# TD3

In [1]:
import numpy as np
from scipy.optimize import fsolve
import pandas as pd

Question 1

In [2]:
def bond_price(freq, coupon_rate, nominal, maturity, y):
    n = int(freq * maturity)
    coupon = coupon_rate * nominal / freq
    times = np.arange(1, n+1) / freq
    return np.sum(coupon / (1+y/freq)**(times*freq)) + nominal / (1+y/freq)**(maturity*freq)


def solve_ytm(price, freq, coupon_rate, nominal, maturity):
    f = lambda y: bond_price(freq, coupon_rate, nominal, maturity, y) - price
    y_guess = 0.01  
    ytm = fsolve(f, y_guess)[0]
    return ytm

y_100 = solve_ytm(100, 4, 0.01, 100, 3)
y_99 = solve_ytm(99, 4, 0.01, 100, 3)

print(y_100)
print(y_99)

0.01
0.013406396766022628


In [3]:
def ComputeDuration(maturity, coupon, y, nominal, price):
    sum_ = 0
    t = 0.25
    while(t < maturity):
        sum_ += (t * coupon*nominal/4) / pow(1+y, t)
        t += 0.25
    
    sum_ += (maturity * (coupon*nominal/4+nominal)) / pow(1+y, maturity)
    return sum_ / price

duration_100 = ComputeDuration(3, 0.01, y_100, 100, 100)
duration_99 = ComputeDuration(3, 0.01, y_99, 100, 99)

print("duration 100 :", duration_100)
print("duration 99 :", duration_99)

def ModifiedDuration(duration, y):
    return duration / (1 + y)

print(ModifiedDuration(duration_100, y_100))
print(ModifiedDuration(duration_99, y_99))

def Convexity(maturity, coupon, y, nominal, price):
    t = 0.25
    sum_ = 0
    while t < maturity:
        sum_ += t * t * (coupon * nominal / 4) / pow(1 + y, t)
        t += 0.25
    sum_ += maturity * maturity * (coupon * nominal / 4 + nominal) / pow(1 + y, maturity)
    return sum_ / price

convexity_100 = Convexity(3, 0.01, y_100, 100, 100)
convexity_99 = Convexity(3, 0.01, y_99, 100, 99)

# CONVEXITY NORMALEMENT 8 VIRGULE QLQCH
print("Convexity 100 : ",convexity_100)
print("Convexity 99 : ",convexity_99)

duration 100 : 2.9595215316483086
duration 99 : 2.959520240340087
2.930219338265652
2.9203686199184182
Convexity 100 :  8.83453817168352
Convexity 99 :  8.834312863072643


Q2.1 – Determine the asset with the highest sensitivity to a spread translation. 

In [4]:
df = pd.DataFrame({"Duration_taux":[2, 0, 4, 3], "Duration_spread": [2, 2.5, 4, 0], "Convexity_taux":[4.5, 0, 16, 10], "Convexity_spread":[4.5, 9, 16, 0], "Spread":[0.015, 0.014, 0.02, 0]})
df.rename(index={0:"Bond1", 1:"FRN", 2:"Bond2", 3:"Govies"}, inplace = True)
df

,Duration_taux,Duration_spread,Convexity_taux,Convexity_spread,Spread
Bond1,2,2.0,4.5,4.5,0.015
FRN,0,2.5,0.0,9.0,0.014
Bond2,4,4.0,16.0,16.0,0.020
Govies,3,0.0,10.0,0.0,0.000


In [5]:
df["DTS"] = df["Duration_spread"] * df["Spread"]
df

,Duration_taux,Duration_spread,Convexity_taux,Convexity_spread,Spread,DTS
Bond1,2,2.0,4.5,4.5,0.015,0.030
FRN,0,2.5,0.0,9.0,0.014,0.035
Bond2,4,4.0,16.0,16.0,0.020,0.080
Govies,3,0.0,10.0,0.0,0.000,0.000


Q2.2 Put this asset in your portfolio with other assets in order to hedge its rate exposure, while 
respecting the budget constraint. 

In [6]:
A = np.array([list(df.drop("FRN")["Duration_taux"]), list(df.drop("FRN")["Convexity_taux"]), [1, 1, 1]])
A_inv = np.linalg.inv(A)
weight = A_inv @ [[0], [0], [0.01]] 
print(weight)

[[ 0.16]
 [ 0.13]
 [-0.28]]


Q2.3 – What is the performance of your strategy (approximated at order 1) if the relative variation 
of spreads is -1% and the absolute variation of rates is 0.1%? 

In [7]:
delta_s = -0.01
delta_r = 0.001
strategy_performance = np.array(-df.drop("FRN")["Duration_taux"] * delta_r - df.drop("FRN")["DTS"] * delta_s) @ weight
print(strategy_performance)

[0.000152]


# Exercise 2

In [9]:
df = pd.read_csv("Lab1_Data.csv", sep = ";")
df["date"] = pd.to_datetime(df["date"], format="%d/%m/%Y")
df.sort_values("date", inplace = True)
df["value"] = (df["value"].astype(str).str.replace(",", ".", regex=False))
df["value"] = pd.to_numeric(df["value"], errors="coerce")
df["return"] = df["value"] / df["value"].shift(1) - 1
df.dropna(inplace=True)

In [26]:
# loss
loss = list(df[df["return"] < 0]["return"] * -1)
loss.sort()
n = len(loss)
e_loss = np.log((loss[int(n-1 - np.floor(np.log(n)) + 1)] - loss[int(n-1 - 2 * np.floor(np.log(n)) + 1)]) / (loss[int(n-1 - 2 * np.floor(np.log(n)) + 1)] - loss[int(n-1 - 4 * np.floor(np.log(n)) + 1)]))
e_loss /= np.log(2)
print("e_loss :", e_loss)

# gain
gain = list(df[df["return"] > 0]["return"] * -1)
gain.sort()
n = len(gain)
e_gain = np.log((gain[int(n-1 - np.floor(np.log(n)) + 1)] - gain[int(n-1 - 2 * np.floor(np.log(n)) + 1)]) / (gain[int(n-1 - 2 * np.floor(np.log(n)) + 1)] - gain[int(n-1 - 4 * np.floor(np.log(n)) + 1)]))
e_gain /= np.log(2)
print("e_gain :", e_gain)

e_loss : -0.5089715779341932
e_gain : -0.9367427921518837
